In [1]:
#Importaciones basicas
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
# graficos incrustados
%matplotlib inline

#Carga de los CSV
BBVA_data = pd.read_csv('train_clientes.csv', header=0)
print("Data Train: ", BBVA_data.shape)
BBVA_test = pd.read_csv('test_clientes.csv', header=0)
print("Data Test:  ", BBVA_test.shape)

Data Train:  (70000, 53)
Data Test:   (30000, 52)


In [2]:
if BBVA_data.isnull().any().any():
    BBVA_data = BBVA_data.dropna()
    print("Clean Data Train: ", BBVA_data.shape)
    

Clean Data Train:  (56262, 53)


In [3]:
if BBVA_test.isnull().any().any():
    tipos = BBVA_test.columns.to_series().groupby(BBVA_test.dtypes).groups
    ctext = tipos[np.dtype('object')]
    columnas = BBVA_test.columns  # lista de todas las columnas
    cnum = list(set(columnas) - set(ctext))
    #COmpletando los valores nulos de colummnas numericas con la media
    for c in cnum:
        mean = BBVA_test[c].mean()
        BBVA_test[c] = BBVA_test[c].fillna(mean)
    # Completando valores nulos de columnas categóricas con la moda
    for c in ctext:
        mode = BBVA_test[c].mode()[0]
        BBVA_test[c] = BBVA_test[c].fillna(mode)
    print("Clean Data Test: ", BBVA_test.shape, BBVA_test.isnull().any().any())
    

Clean Data Test:  (30000, 52) False


In [4]:
def factorizacion_data(datus):
    #Pasando los datos de edad a rango de 10 años
    EDAD2 = pd.cut(datus['EDAD'], range(0, 150, 10))
    datus['EDAD'] = EDAD2
    
    #Factorizando la columna EDAD
    datus['EDAD'], _ = pd.factorize(datus['EDAD'])
    
    #Factorizar las demas columnas de datos categoricos
    tipos = datus.columns.to_series().groupby(datus.dtypes).groups
    ctext = tipos[np.dtype('object')]
    for c in ctext:
        datus[c], _ = pd.factorize(datus[c])
    print(type(datus), datus.shape)
    return datus

BBVA_data = factorizacion_data(BBVA_data)
BBVA_test = factorizacion_data(BBVA_test)

<class 'pandas.core.frame.DataFrame'> (56262, 53)
<class 'pandas.core.frame.DataFrame'> (30000, 52)


In [5]:
print(BBVA_data.isnull().any().any())
print(BBVA_test.isnull().any().any())

False
False


In [6]:
X_train = BBVA_data.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1)
print(type(X_train), X_train.shape)
X_predic = BBVA_test.drop(['ID_CORRELATIVO'], axis=1)
print(type(X_predic), X_predic.shape)
y_train = BBVA_data['ATTRITION'].values
print(type(y_train), y_train.shape)

<class 'pandas.core.frame.DataFrame'> (56262, 51)
<class 'pandas.core.frame.DataFrame'> (30000, 51)
<class 'numpy.ndarray'> (56262,)


In [7]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE

columnas = list(X_train.columns.values)
era = RFE(ExtraTreesClassifier(), 20)  # número de atributos a seleccionar
era = era.fit(X_train.values, y_train)
# imprimir resultados
atrib = era.support_
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
print(atributos)

['RANG_INGRESO', 'EDAD', 'ANTIGUEDAD', 'RANG_SDO_PASIVO_MENOS0', 'SDO_ACTIVO_MENOS0', 'RANG_NRO_PRODUCTOS_MENOS0', 'NRO_ACCES_CANAL1_MENOS0', 'NRO_ACCES_CANAL1_MENOS1', 'NRO_ACCES_CANAL2_MENOS0', 'NRO_ACCES_CANAL3_MENOS0', 'NRO_ACCES_CANAL3_MENOS1', 'NRO_ACCES_CANAL3_MENOS2', 'NRO_ACCES_CANAL3_MENOS3', 'NRO_ACCES_CANAL3_MENOS4', 'NRO_ACCES_CANAL3_MENOS5', 'NRO_ENTID_SSFF_MENOS0', 'NRO_ENTID_SSFF_MENOS1', 'NRO_ENTID_SSFF_MENOS3', 'NRO_ENTID_SSFF_MENOS4', 'NRO_ENTID_SSFF_MENOS5']


In [7]:
atributos = ['RANG_INGRESO', 'EDAD', 'ANTIGUEDAD', 'RANG_SDO_PASIVO_MENOS0', 'SDO_ACTIVO_MENOS0', 'RANG_NRO_PRODUCTOS_MENOS0', 'NRO_ACCES_CANAL1_MENOS0', 'NRO_ACCES_CANAL1_MENOS1', 'NRO_ACCES_CANAL2_MENOS0', 'NRO_ACCES_CANAL3_MENOS0', 'NRO_ACCES_CANAL3_MENOS1', 'NRO_ACCES_CANAL3_MENOS2', 'NRO_ACCES_CANAL3_MENOS3', 'NRO_ACCES_CANAL3_MENOS4', 'NRO_ACCES_CANAL3_MENOS5', 'NRO_ENTID_SSFF_MENOS0', 'NRO_ENTID_SSFF_MENOS1', 'NRO_ENTID_SSFF_MENOS3', 'NRO_ENTID_SSFF_MENOS4', 'NRO_ENTID_SSFF_MENOS5']
print(type(X_train), type(X_predic))
xx_train = X_train.loc[:,atributos].values
print(type(X_train), X_train.shape)
xx_predic = X_predic.loc[:,atributos].values
print(type(X_predic), X_predic.shape)

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> (56262, 51)
<class 'pandas.core.frame.DataFrame'> (30000, 51)


In [10]:
print(type(xx_train), type(y_train))
from imblearn.over_sampling import SMOTE
X_smote, y_smote = SMOTE(kind='svm').fit_sample(xx_train, y_train)

print("X_smote:: ", X_smote.shape)
print("y_smote:: ", y_smote.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
X_smote::  (98891, 20)
y_smote::  (98891,)


In [11]:
from sklearn.preprocessing import StandardScaler
X_smote = StandardScaler().fit_transform(X_smote.astype(float))
X_smote.shape

(98891, 20)

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=25)
#Construimos el modelo sobre los datos de entrenamiento
clf.fit(X_smote,y_smote)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [13]:
prediccion = clf.predict(xx_predic)
print(prediccion.shape)
prediccion_proba = clf.predict_proba(xx_predic)
print(prediccion_proba.shape)

(30000,)
(30000, 2)


In [14]:
print(prediccion)
print("//////////////")
print(prediccion_proba)
print("//////////////")
result = [float("{0:.2f}".format(_)) for _ in list(zip(*prediccion_proba))[1]]
print(result)


[1 1 1 ..., 1 0 1]
//////////////
[[ 0.04  0.96]
 [ 0.04  0.96]
 [ 0.44  0.56]
 ..., 
 [ 0.36  0.64]
 [ 0.6   0.4 ]
 [ 0.48  0.52]]
//////////////
[0.96, 0.96, 0.56, 0.72, 0.88, 0.52, 0.6, 0.48, 0.72, 0.56, 0.24, 0.56, 0.44, 0.8, 0.4, 0.68, 0.68, 0.72, 0.56, 0.4, 0.48, 0.76, 0.48, 0.4, 0.8, 0.8, 0.4, 0.72, 0.56, 0.4, 0.68, 0.72, 0.6, 0.64, 0.84, 0.68, 0.76, 0.44, 0.68, 0.4, 0.88, 0.56, 0.48, 0.48, 0.6, 0.84, 0.6, 0.68, 0.56, 0.96, 0.52, 0.64, 0.52, 0.84, 0.56, 0.2, 0.48, 0.4, 0.64, 0.44, 0.52, 0.68, 0.32, 0.8, 0.64, 0.76, 0.72, 0.36, 0.52, 0.48, 0.84, 0.56, 0.68, 0.48, 0.72, 0.8, 0.32, 0.44, 0.64, 0.84, 0.48, 0.6, 1.0, 0.28, 0.56, 0.68, 0.48, 0.36, 0.44, 0.48, 0.68, 0.56, 0.56, 0.48, 0.52, 0.72, 0.64, 0.44, 0.68, 0.4, 0.8, 0.28, 0.6, 0.84, 0.64, 0.52, 0.76, 0.68, 0.76, 0.48, 0.52, 0.88, 0.68, 0.84, 0.36, 0.92, 0.48, 0.64, 0.56, 0.64, 0.64, 0.52, 0.48, 0.56, 0.72, 0.44, 0.88, 0.48, 0.6, 0.32, 0.32, 0.52, 0.8, 0.72, 0.4, 0.64, 0.32, 0.68, 0.68, 0.88, 0.56, 0.56, 0.28, 0.8, 0.32, 0.52, 0.

In [15]:
sum(prediccion)

21119

In [16]:
idd = BBVA_test['ID_CORRELATIVO']
idd.shape

(30000,)

In [17]:
resultados = list(zip(list(idd), result))
resultados

[(47411, 0.96),
 (39861, 0.96),
 (38898, 0.56),
 (50927, 0.72),
 (32969, 0.88),
 (89661, 0.52),
 (12197, 0.6),
 (71520, 0.48),
 (59759, 0.72),
 (36743, 0.56),
 (10736, 0.24),
 (5106, 0.56),
 (44506, 0.44),
 (47767, 0.8),
 (74686, 0.4),
 (63023, 0.68),
 (13584, 0.68),
 (317, 0.72),
 (62570, 0.56),
 (68244, 0.4),
 (72881, 0.48),
 (18280, 0.76),
 (70682, 0.48),
 (79902, 0.4),
 (35073, 0.8),
 (84331, 0.8),
 (70921, 0.4),
 (82245, 0.72),
 (17714, 0.56),
 (13993, 0.4),
 (658, 0.68),
 (33916, 0.72),
 (92737, 0.6),
 (22844, 0.64),
 (93339, 0.84),
 (29572, 0.68),
 (24751, 0.76),
 (37862, 0.44),
 (26676, 0.68),
 (8675, 0.4),
 (64245, 0.88),
 (57248, 0.56),
 (3531, 0.48),
 (9834, 0.48),
 (74246, 0.6),
 (56683, 0.84),
 (16847, 0.6),
 (96617, 0.68),
 (94167, 0.56),
 (42800, 0.96),
 (70106, 0.52),
 (42486, 0.64),
 (78076, 0.52),
 (85819, 0.84),
 (15786, 0.56),
 (61483, 0.2),
 (25630, 0.48),
 (22501, 0.4),
 (92447, 0.64),
 (13795, 0.44),
 (57101, 0.52),
 (55077, 0.68),
 (47600, 0.32),
 (5924, 0.8),
 

In [18]:
import csv
resultados.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
resultados

[('ID_CORRELATIVO', 'ATTRITION'),
 (47411, 0.96),
 (39861, 0.96),
 (38898, 0.56),
 (50927, 0.72),
 (32969, 0.88),
 (89661, 0.52),
 (12197, 0.6),
 (71520, 0.48),
 (59759, 0.72),
 (36743, 0.56),
 (10736, 0.24),
 (5106, 0.56),
 (44506, 0.44),
 (47767, 0.8),
 (74686, 0.4),
 (63023, 0.68),
 (13584, 0.68),
 (317, 0.72),
 (62570, 0.56),
 (68244, 0.4),
 (72881, 0.48),
 (18280, 0.76),
 (70682, 0.48),
 (79902, 0.4),
 (35073, 0.8),
 (84331, 0.8),
 (70921, 0.4),
 (82245, 0.72),
 (17714, 0.56),
 (13993, 0.4),
 (658, 0.68),
 (33916, 0.72),
 (92737, 0.6),
 (22844, 0.64),
 (93339, 0.84),
 (29572, 0.68),
 (24751, 0.76),
 (37862, 0.44),
 (26676, 0.68),
 (8675, 0.4),
 (64245, 0.88),
 (57248, 0.56),
 (3531, 0.48),
 (9834, 0.48),
 (74246, 0.6),
 (56683, 0.84),
 (16847, 0.6),
 (96617, 0.68),
 (94167, 0.56),
 (42800, 0.96),
 (70106, 0.52),
 (42486, 0.64),
 (78076, 0.52),
 (85819, 0.84),
 (15786, 0.56),
 (61483, 0.2),
 (25630, 0.48),
 (22501, 0.4),
 (92447, 0.64),
 (13795, 0.44),
 (57101, 0.52),
 (55077, 0.68

In [19]:
with open("output_20_var_smote_svm.csv", 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in resultados:
           wr.writerow(row)

In [8]:
#Probando con data no balanceada:::::
print("xx_train:: ", xx_train.shape, type(xx_train))
print("y_train:: ", y_train.shape, type(y_train))

xx_train::  (56262, 20) <class 'numpy.ndarray'>
y_train::  (56262,) <class 'numpy.ndarray'>


In [10]:
from sklearn.preprocessing import StandardScaler
xx_train_std = StandardScaler().fit_transform(xx_train.astype(float))
xx_train_std.shape

(56262, 20)

In [11]:
from sklearn.ensemble import RandomForestClassifier
clf_no_b = RandomForestClassifier(n_estimators=25)
#Construimos el modelo sobre los datos de entrenamiento
clf_no_b.fit(xx_train_std,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [12]:
prediccion_no_b = clf_no_b.predict(xx_predic)
print(prediccion_no_b.shape)
prediccion_proba_no_b = clf_no_b.predict_proba(xx_predic)
print(prediccion_proba_no_b.shape)

(30000,)
(30000, 2)


In [13]:
sum(prediccion_no_b)

25

In [14]:
print(prediccion_no_b)
print("//////////////")
print(prediccion_proba_no_b)
print("//////////////")
result_no_b = [float("{0:.2f}".format(_)) for _ in list(zip(*prediccion_proba_no_b))[1]]
print(result_no_b)

[0 0 0 ..., 0 0 0]
//////////////
[[ 0.92  0.08]
 [ 0.92  0.08]
 [ 0.84  0.16]
 ..., 
 [ 0.88  0.12]
 [ 1.    0.  ]
 [ 0.72  0.28]]
//////////////
[0.08, 0.08, 0.16, 0.08, 0.12, 0.2, 0.2, 0.28, 0.04, 0.04, 0.12, 0.2, 0.24, 0.04, 0.32, 0.0, 0.12, 0.04, 0.24, 0.12, 0.2, 0.12, 0.28, 0.0, 0.12, 0.2, 0.2, 0.12, 0.16, 0.08, 0.16, 0.24, 0.16, 0.12, 0.2, 0.28, 0.04, 0.2, 0.2, 0.44, 0.24, 0.04, 0.12, 0.0, 0.04, 0.0, 0.08, 0.08, 0.16, 0.16, 0.24, 0.2, 0.12, 0.12, 0.12, 0.28, 0.44, 0.16, 0.08, 0.28, 0.2, 0.32, 0.04, 0.16, 0.08, 0.24, 0.2, 0.08, 0.16, 0.08, 0.0, 0.32, 0.04, 0.2, 0.16, 0.0, 0.04, 0.16, 0.04, 0.12, 0.12, 0.32, 0.08, 0.16, 0.2, 0.08, 0.2, 0.12, 0.16, 0.16, 0.16, 0.0, 0.04, 0.16, 0.24, 0.24, 0.0, 0.12, 0.2, 0.28, 0.0, 0.12, 0.12, 0.16, 0.2, 0.04, 0.04, 0.36, 0.04, 0.24, 0.0, 0.16, 0.36, 0.16, 0.08, 0.08, 0.32, 0.12, 0.04, 0.2, 0.28, 0.12, 0.0, 0.04, 0.24, 0.28, 0.12, 0.12, 0.12, 0.32, 0.0, 0.2, 0.12, 0.12, 0.32, 0.12, 0.24, 0.04, 0.08, 0.0, 0.2, 0.2, 0.0, 0.0, 0.08, 0.24, 0.12, 0.28, 

In [15]:
idd = BBVA_test['ID_CORRELATIVO']
idd.shape

(30000,)

In [17]:
resultados_no_b = list(zip(list(idd), result_no_b))
resultados_no_b

[(47411, 0.08),
 (39861, 0.08),
 (38898, 0.16),
 (50927, 0.08),
 (32969, 0.12),
 (89661, 0.2),
 (12197, 0.2),
 (71520, 0.28),
 (59759, 0.04),
 (36743, 0.04),
 (10736, 0.12),
 (5106, 0.2),
 (44506, 0.24),
 (47767, 0.04),
 (74686, 0.32),
 (63023, 0.0),
 (13584, 0.12),
 (317, 0.04),
 (62570, 0.24),
 (68244, 0.12),
 (72881, 0.2),
 (18280, 0.12),
 (70682, 0.28),
 (79902, 0.0),
 (35073, 0.12),
 (84331, 0.2),
 (70921, 0.2),
 (82245, 0.12),
 (17714, 0.16),
 (13993, 0.08),
 (658, 0.16),
 (33916, 0.24),
 (92737, 0.16),
 (22844, 0.12),
 (93339, 0.2),
 (29572, 0.28),
 (24751, 0.04),
 (37862, 0.2),
 (26676, 0.2),
 (8675, 0.44),
 (64245, 0.24),
 (57248, 0.04),
 (3531, 0.12),
 (9834, 0.0),
 (74246, 0.04),
 (56683, 0.0),
 (16847, 0.08),
 (96617, 0.08),
 (94167, 0.16),
 (42800, 0.16),
 (70106, 0.24),
 (42486, 0.2),
 (78076, 0.12),
 (85819, 0.12),
 (15786, 0.12),
 (61483, 0.28),
 (25630, 0.44),
 (22501, 0.16),
 (92447, 0.08),
 (13795, 0.28),
 (57101, 0.2),
 (55077, 0.32),
 (47600, 0.04),
 (5924, 0.16),


In [18]:
import csv
resultados_no_b.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
resultados_no_b

[('ID_CORRELATIVO', 'ATTRITION'),
 (47411, 0.08),
 (39861, 0.08),
 (38898, 0.16),
 (50927, 0.08),
 (32969, 0.12),
 (89661, 0.2),
 (12197, 0.2),
 (71520, 0.28),
 (59759, 0.04),
 (36743, 0.04),
 (10736, 0.12),
 (5106, 0.2),
 (44506, 0.24),
 (47767, 0.04),
 (74686, 0.32),
 (63023, 0.0),
 (13584, 0.12),
 (317, 0.04),
 (62570, 0.24),
 (68244, 0.12),
 (72881, 0.2),
 (18280, 0.12),
 (70682, 0.28),
 (79902, 0.0),
 (35073, 0.12),
 (84331, 0.2),
 (70921, 0.2),
 (82245, 0.12),
 (17714, 0.16),
 (13993, 0.08),
 (658, 0.16),
 (33916, 0.24),
 (92737, 0.16),
 (22844, 0.12),
 (93339, 0.2),
 (29572, 0.28),
 (24751, 0.04),
 (37862, 0.2),
 (26676, 0.2),
 (8675, 0.44),
 (64245, 0.24),
 (57248, 0.04),
 (3531, 0.12),
 (9834, 0.0),
 (74246, 0.04),
 (56683, 0.0),
 (16847, 0.08),
 (96617, 0.08),
 (94167, 0.16),
 (42800, 0.16),
 (70106, 0.24),
 (42486, 0.2),
 (78076, 0.12),
 (85819, 0.12),
 (15786, 0.12),
 (61483, 0.28),
 (25630, 0.44),
 (22501, 0.16),
 (92447, 0.08),
 (13795, 0.28),
 (57101, 0.2),
 (55077, 0.32

In [20]:
with open("output_20_var_smote_svm_no_balanceada.csv", 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in resultados_no_b:
           wr.writerow(row)

In [21]:
#Probando con todos los campos / sin reducir por relevancia ni balancear
X_all = X_train.values
print("X_all:: ", X_all.shape, type(X_all))
print("y_train:: ", y_train.shape, type(y_train))

X_all::  (56262, 51) <class 'numpy.ndarray'>
y_train::  (56262,) <class 'numpy.ndarray'>


In [22]:
from sklearn.preprocessing import StandardScaler
xx_train_all = StandardScaler().fit_transform(X_all.astype(float))
xx_train_all.shape

(56262, 51)

In [23]:
from sklearn.ensemble import RandomForestClassifier
clf_all = RandomForestClassifier(n_estimators=25)
#Construimos el modelo sobre los datos de entrenamiento
clf_all.fit(xx_train_all,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
xx_predic_all = X_predic.values
prediccion_all = clf_all.predict(xx_predic_all)
print(prediccion_all.shape)
prediccion_proba_all = clf_all.predict_proba(xx_predic_all)
print(prediccion_proba_all.shape)

(30000,)
(30000, 2)


In [26]:
sum(prediccion_all)

344

In [27]:
print(prediccion_all)
print("//////////////")
print(prediccion_proba_all)
print("//////////////")
result_all = [float("{0:.2f}".format(_)) for _ in list(zip(*prediccion_proba_all))[1]]
print(result_all)

[0 0 0 ..., 0 0 0]
//////////////
[[ 1.    0.  ]
 [ 0.96  0.04]
 [ 0.72  0.28]
 ..., 
 [ 0.76  0.24]
 [ 0.96  0.04]
 [ 0.64  0.36]]
//////////////
[0.0, 0.04, 0.28, 0.16, 0.08, 0.12, 0.16, 0.48, 0.0, 0.0, 0.24, 0.2, 0.12, 0.0, 0.24, 0.04, 0.2, 0.0, 0.12, 0.16, 0.28, 0.08, 0.56, 0.0, 0.2, 0.2, 0.28, 0.2, 0.32, 0.2, 0.16, 0.24, 0.08, 0.2, 0.08, 0.56, 0.0, 0.4, 0.12, 0.28, 0.12, 0.08, 0.2, 0.04, 0.0, 0.04, 0.0, 0.16, 0.04, 0.04, 0.12, 0.08, 0.24, 0.04, 0.08, 0.08, 0.4, 0.2, 0.08, 0.28, 0.28, 0.12, 0.04, 0.16, 0.04, 0.32, 0.2, 0.08, 0.2, 0.12, 0.04, 0.12, 0.08, 0.36, 0.2, 0.0, 0.12, 0.24, 0.44, 0.08, 0.24, 0.08, 0.0, 0.24, 0.28, 0.04, 0.24, 0.16, 0.2, 0.08, 0.24, 0.0, 0.0, 0.2, 0.2, 0.04, 0.04, 0.28, 0.48, 0.12, 0.04, 0.24, 0.04, 0.04, 0.12, 0.04, 0.04, 0.16, 0.04, 0.2, 0.24, 0.16, 0.16, 0.08, 0.36, 0.04, 0.08, 0.16, 0.08, 0.32, 0.28, 0.08, 0.04, 0.0, 0.08, 0.52, 0.16, 0.12, 0.28, 0.44, 0.0, 0.24, 0.12, 0.0, 0.16, 0.08, 0.08, 0.08, 0.04, 0.0, 0.36, 0.2, 0.0, 0.04, 0.08, 0.16, 0.32, 0.28, 0

In [29]:
idd = BBVA_test['ID_CORRELATIVO']
print("idd", idd.shape)
resultados_all = list(zip(list(idd), result_all))
print("resultados_all", len(resultados_all))

idd (30000,)
resultados_all 30000


In [30]:
import csv
resultados_all.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
resultados_all

[('ID_CORRELATIVO', 'ATTRITION'),
 (47411, 0.0),
 (39861, 0.04),
 (38898, 0.28),
 (50927, 0.16),
 (32969, 0.08),
 (89661, 0.12),
 (12197, 0.16),
 (71520, 0.48),
 (59759, 0.0),
 (36743, 0.0),
 (10736, 0.24),
 (5106, 0.2),
 (44506, 0.12),
 (47767, 0.0),
 (74686, 0.24),
 (63023, 0.04),
 (13584, 0.2),
 (317, 0.0),
 (62570, 0.12),
 (68244, 0.16),
 (72881, 0.28),
 (18280, 0.08),
 (70682, 0.56),
 (79902, 0.0),
 (35073, 0.2),
 (84331, 0.2),
 (70921, 0.28),
 (82245, 0.2),
 (17714, 0.32),
 (13993, 0.2),
 (658, 0.16),
 (33916, 0.24),
 (92737, 0.08),
 (22844, 0.2),
 (93339, 0.08),
 (29572, 0.56),
 (24751, 0.0),
 (37862, 0.4),
 (26676, 0.12),
 (8675, 0.28),
 (64245, 0.12),
 (57248, 0.08),
 (3531, 0.2),
 (9834, 0.04),
 (74246, 0.0),
 (56683, 0.04),
 (16847, 0.0),
 (96617, 0.16),
 (94167, 0.04),
 (42800, 0.04),
 (70106, 0.12),
 (42486, 0.08),
 (78076, 0.24),
 (85819, 0.04),
 (15786, 0.08),
 (61483, 0.08),
 (25630, 0.4),
 (22501, 0.2),
 (92447, 0.08),
 (13795, 0.28),
 (57101, 0.28),
 (55077, 0.12),
 (

In [31]:
with open("output_all_var_no_balanceada.csv", 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in resultados_all:
           wr.writerow(row)